# Machine Learning Pipelines

A Machine Learning pipeline is a set of independent steps that each can have different compute, data, environment, etc. dependencies. Each step is treated as a separate run and all of the four aspects of a run (metrics, logs, snapshot and outputs) are stored independently. After the pipeline is created, it can be published as an API for later reuse.

This gives you huge benefit as you have ultimate control over defining each step of the pipeline in a way that serves best for that particular step.

ML Pipelines are some type of workflow orchestration tools. The main difference between ML Pipelines and regular job workflows (such as Azure Data Factory or Airflow) is that this workflow engine is designed to address ML needs. Therefore, if you have non-ML related workflow it's recommended you use generic workflow engines. You may call an ML Pipeline from a generic pipeline as a step.

![ML Pipeline](assets/MLOps_Pipeline.jpg)

## Prerequisites and Azure Machine Learning Basics
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration Notebook](https://aka.ms/pl-config) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc. 

## Azure Machine Learning and Pipeline SDK-specific imports

In [ ]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.exceptions import ComputeTargetException
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

import os
import shutil
import urllib
import numpy as np
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


## Initialize workspace

Initialize a workspace object from persisted configuration. If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure the config file is present at .\config.json

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create an Azure ML experiment
Let's create an experiment named "tf-mnist" and a folder to hold the training scripts. 

> The best practice is to use separate folders for scripts and its dependent files for each step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step. 

> The script runs will be recorded under the experiment in Azure.

In [ ]:
script_folder = './project-solder-pipelines'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='Hyperdrive_sample')

## Download MNIST dataset
In order to train on the MNIST dataset we will first need to download it from Yan LeCun's web site directly and save them in a `data` folder locally.

In [ ]:
data_folder = os.path.join(os.getcwd(), 'data/mnist')
os.makedirs(data_folder, exist_ok=True)

urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
                           filename=os.path.join(data_folder, 'train-images.gz'))
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
                           filename=os.path.join(data_folder, 'train-labels.gz'))
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
                           filename=os.path.join(data_folder, 'test-images.gz'))
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz',
                           filename=os.path.join(data_folder, 'test-labels.gz'))

## Show some sample images
Let's load the downloaded compressed file into numpy arrays using some utility functions included in the `utils.py` library file from the current folder. Then we use `matplotlib` to plot 30 random images from the dataset along with their labels.

In [ ]:
from utils import load_data

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the neural network converge faster.
X_train = load_data(os.path.join(data_folder, 'train-images.gz'), False) / np.float32(255.0)
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / np.float32(255.0)
y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True).reshape(-1)
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)


count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = 10, y = -10, s = y_train[i], fontsize = 18)
    plt.imshow(X_train[i].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

## Upload MNIST dataset to blob datastore 
A [datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data) is a place where data can be stored that is then made accessible to a Run either by means of mounting or copying the data to the compute target. In the next step, we will use Azure Blob Storage and upload the training and test set into the Azure Blob datastore, which we will then later be mount on a Batch AI cluster for training.

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./data/mnist', target_path='mnist', overwrite=True, show_progress=True)

## Create Azure Machine Learning datasets
By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

In [ ]:
dataset = Dataset.File.from_files(datastore.path('mnist'))
dataset.register(name='mnist', workspace=ws)

## Retrieve or create a Azure Machine Learning compute
Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Azure Machine Learning Compute in the current workspace, if it doesn't already exist. We will then run the training script on this compute target.

If we could not find the compute with the given name in the previous cell, then we will create a new compute here. This process is broken down into the following steps:

1. Create the configuration
2. Create the Azure Machine Learning compute

**This process will take a few minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.**


In [ ]:
cluster_name = "v100-1-g-1-n"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target {}.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                               max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

print("Azure Machine Learning Compute attached")

cpu_cluster_name = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

## Retrieve an Environment
In this tutorial, we will use one of Azure ML's curated TensorFlow environments for training. Curated environments are available in your workspace by default. Specifically, we will use the TensorFlow 2.0 GPU curated environment.

In [ ]:
tf_env = Environment.get(ws, name='AzureML-TensorFlow-2.0-GPU')

### Setup an input for the ScriptRunConfig step
You can mount dataset to remote compute.

In [ ]:
data_folder = dataset.as_mount()

## Configure the training job
Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on

In [ ]:
src = ScriptRunConfig(source_directory=script_folder,
                      script='tf_mnist.py',
                      arguments=['--data-folder', data_folder],
                      compute_target=compute_target,
                      environment=tf_env)

## Intelligent hyperparameter tuning
Now let's try hyperparameter tuning by launching multiple runs on the cluster. First let's define the parameter space using random sampling.

In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, the best validation accuracy (`validation_acc`).

In [ ]:
ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 200, 500),
        '--learning-rate': loguniform(-6, -1)
    }
)

Now we will define an early termnination policy. The `BanditPolicy` basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.

Refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-tune-hyperparameters#specify-an-early-termination-policy) for more information on the BanditPolicy and other policies available.

In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

Now we are ready to configure a run configuration object, and specify the primary metric `validation_acc` that's recorded in your training runs. If you go back to visit the training script, you will notice that this value is being logged after every epoch (a full batch set). We also want to tell the service that we are looking to maximizing this value. We also set the number of samples to 20, and maximal concurrent job to 4, which is the same as the number of nodes in our computer cluster.

In [ ]:
hd_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='validation_acc', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=4,
                             max_concurrent_runs=4)


### HyperDriveStep
HyperDriveStep can be used to run HyperDrive job as a step in pipeline.
- **name:** Name of the step
- **hyperdrive_config:** A HyperDriveConfig that defines the configuration for this HyperDrive run
- **inputs:** List of input port bindings
- **outputs:** List of output port bindings
- **metrics_output:** Optional value specifying the location to store HyperDrive run metrics as a JSON file
- **allow_reuse:** whether to allow reuse
- **version:** version


In [ ]:
metrics_output_name = 'metrics_output'
metrics_data = PipelineData(name='metrics_data',
                            datastore=datastore,
                            pipeline_output_name=metrics_output_name,
                            training_output=TrainingOutput("Metrics"))

model_output_name = 'model_output'
saved_model = PipelineData(name='saved_model',
                            datastore=datastore,
                            pipeline_output_name=model_output_name,
                            training_output=TrainingOutput("Model",
                                                           model_file="outputs/model/saved_model.pb"))

hd_step_name='hd_step01'
hd_step = HyperDriveStep(
    name=hd_step_name,
    hyperdrive_config=hd_config,
    inputs=[data_folder],
    outputs=[metrics_data, saved_model])

## Find and register best model
When all the jobs finish, we can choose to register the model that has the highest accuracy through an additional PythonScriptStep.

Through this additional register_model_step, we register the chosen files as a model named `tf-dnn-mnist` under the workspace for deployment.

In [ ]:
conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-sdk")

rcfg = RunConfiguration(conda_dependencies=conda_dep)

register_model_step = PythonScriptStep(source_directory=script_folder,
                                       script_name='register_model.py',
                                       name="register_model_step01",
                                       inputs=[saved_model],
                                       compute_target=cpu_cluster,
                                       arguments=["--saved-model", saved_model],
                                       allow_reuse=True,
                                       runconfig=rcfg)

register_model_step.run_after(hd_step)

### Run the pipeline

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[hd_step, register_model_step])
pipeline_run = exp.submit(pipeline)

### Monitor using widget

In [ ]:
# # To get the historical run
# from azureml.pipeline.core.run import PipelineRun
# 
# pipeline_run = PipelineRun(exp, run_id="c879b1a3-2c7e-4da4-8484-80fe411ee449")

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

### Wait for the completion of this Pipeline run

In [ ]:
pipeline_run.wait_for_completion()

### Retrieve the metrics
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will show the result metrics.

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import pandas as pd
import json
with open(metrics_output._path_on_datastore) as f:  
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

## Databricks Step in the Pipeline

### Retrieve Databricks Compute

In [ ]:
from azureml.pipeline.steps import DatabricksStep
from azureml.core.compute import DatabricksCompute

# Replace with your account info before running.
 
db_compute_name=os.getenv("DATABRICKS_COMPUTE_NAME", "databricks-demo") # Databricks compute name

try:
    databricks_compute = DatabricksCompute(workspace=ws, name=db_compute_name)
    print('Compute target {} already exists'.format(db_compute_name))
except ComputeTargetException:
    print('Compute not found, it needs to be attached')

In [ ]:
# notebook_path=os.getenv("DATABRICKS_NOTEBOOK_PATH", "<my-databricks-notebook-path>") # Databricks notebook path

dbNbStep = DatabricksStep(
    name="ADBScript",
    source_directory='databricks-step/',
    python_script_name='spark-test.py',
    existing_cluster_id = '0519-025444-grams615',
    # num_workers=1,
    # pypi_libraries=['azureml-sdk'],
    run_name='DB_Pipeline_demo',
    compute_target=databricks_compute,
    allow_reuse=False
)

In [ ]:
dbNbStep2 = DatabricksStep(
    name="ADBScript",
    source_directory='databricks-step/',
    python_script_name='spark-test.py',
    existing_cluster_id = '0519-025444-grams615',
    # num_workers=1,
    # pypi_libraries=['azureml-sdk'],
    run_name='DB_Pipeline_demo',
    compute_target=databricks_compute,
    allow_reuse=False
)

In [ ]:
dbNbStep2.run_after(dbNbStep)

In [ ]:
steps = [dbNbStep2]
pipeline = Pipeline(workspace=ws, steps=steps)

In [ ]:
# # To get the historical run
# from azureml.pipeline.core.run import PipelineRun
# 
# pipeline_run = PipelineRun(exp, run_id="c879b1a3-2c7e-4da4-8484-80fe411ee449")

In [ ]:
pipeline_run = Experiment(ws, 'DB_demo').submit(pipeline)

In [ ]:
RunDetails(pipeline_run).show()

In [ ]:
import os
import azureml.core
import pandas as pd
from azureml.core.runconfig import JarLibrary
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import DatabricksStep, PythonScriptStep
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputAbstractDataset

datastore = Datastore(ws, "blobgen2")
print('Datastore {} will be used'.format(datastore.name))

In [ ]:
cluster_name = "cpu-cluster-4"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

In [ ]:
source_directory = "."

isDeltaUsed = False

if isDeltaUsed:
    databricks_script_name = "adb_run_delta.py"
    aml_script_name = 'aml_run_delta.py'
else:
    databricks_script_name = "adb_run.py"
    aml_script_name = 'aml_test.py'

feature_dataset_name = "feature_titanic"

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

tf_env_c = Environment('deltalake')

conda_dep = CondaDependencies()

conda_dep.add_pip_package("sklearn")
conda_dep.add_pip_package("deltalake")
conda_dep.remove_pip_package('azureml-defaults')
conda_dep.add_pip_package('azureml-core')
conda_dep.add_pip_package('pandas')

# Adds dependencies to PythonSection of myenv
tf_env_c.python.conda_dependencies=conda_dep

tf_env_c = tf_env_c.register(workspace=ws)

rcfg = RunConfiguration()
rcfg.environment = tf_env_c

In [ ]:
ds_step_1_output = Dataset.File.from_files(path=[(datastore, '')])

# ds_step_1_output = DataReference(datastore, data_reference_name=None, path_on_datastore=None, mode='mount')

In [ ]:
ds_step_1_output = Dataset.File.from_files(path=[(datastore, '')])

aml_step = PythonScriptStep(script_name=aml_script_name,
                            name="AML Train",
                            source_directory=source_directory,
                            inputs=[ds_step_1_output.as_named_input('data_folder').as_mount()],
                            compute_target=compute_target,
                            arguments=['--data_folder', ds_step_1_output.as_named_input('data_folder2').as_mount(),
                                       '--featureset_name', feature_dataset_name,
                                       '--model_name', 'titanic_model'],
                            allow_reuse=False,
                            runconfig=rcfg)


In [ ]:
aml_step = PythonScriptStep(script_name=aml_script_name,
                            name="AML Train",
                            source_directory=source_directory,
                            inputs=[ds_step_1_output],
                            compute_target=compute_target,
                            arguments=['--data_folder', ds_step_1_output,
                                       '--featureset_name', feature_dataset_name,
                                       '--model_name', 'titanic_model'],
                            allow_reuse=False,
                            runconfig=rcfg)


In [ ]:
steps = [aml_step]
pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = Experiment(ws, 'ADLS_Gen2_Test').submit(pipeline)
